# Operationnal Planning

In [ ]:
import calendar
from datetime import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets
from ipydatetime import NaiveDatetimePicker
from IPython.display import display, clear_output, Markdown

from resourcecode import Client
from resourcecode.opsplanning import ww_calc, wwmonstats, olmonstats, oplen_calc

MONTH_NAMES = list(calendar.month_name)


class SearchAlgo:
    CONTINUOUS = "Continuous windows (ie critical operations)"
    CONCURRENT = "Concurrent windows (ie, non-critical operations)"


class ComputationMode:
    EACH_MONTH = "Each month"
    SPECIFIC_DATE = "Specificy datetime"


client = Client()
data = None
percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]

output = widgets.Output()
output_operation_plot_widgets = widgets.Output()
output_weather_window_plot = widgets.Output()
output_operation_plot = widgets.Output()

In [ ]:
def update_dataframe(change_url):
    global data
    with output:
        clear_output()
        display(Markdown("The new selection is {new}".format(**change_url)))
        display(Markdown("Will re-compute the data end rerender the graphs"))

    data = client.get_dataframe_from_url(change_url["new"], parameters=("hs", "tp"))
    update_plots()


def update_plots(event=None):
    update_weather_window_plot(event)
    update_operation_plot(event)


def update_weather_window_plot(event=None):
    if data is None:
        return
    data_matching_criteria = data.query(criteria_widget.value)
    with output_weather_window_plot:
        clear_output(wait=True)
        display(
            plot_wwmonstats(
                data_matching_criteria,
                winlen=winlen_widget.value,
                concurrent_windows=concurrent_windows_widget.value
                == SearchAlgo.CONCURRENT,
            )
        )


def update_operation_plot(event=None):
    if data is None:
        return

    data_matching_criteria = data.query(criteria_widget.value)
    if computation_mode_widget.value == ComputationMode.EACH_MONTH:
        monstrt = True
        date = startday_widget.value
    else:
        monstrt = False
        date = datetime_widget.value

    with output_operation_plot:
        clear_output(wait=True)
        display(
            plot_olmonstats(
                data_matching_criteria,
                oplen=winlen_widget.value,
                critical_operation=concurrent_windows_widget.value
                == SearchAlgo.CONTINUOUS,
                date=date,
                monstrt=monstrt,
            )
        )

In [ ]:
url_widget = widgets.Text(
    value="",
    placeholder="Paste the URL of your selection",
    description="Selection:",
    continuous_update=False,
)

criteria_widget = widgets.Text(
    value="hs < 2 and tp < 3",
    description="Operational criteria:",
    continuous_update=False,
)

concurrent_windows_widget = widgets.RadioButtons(
    options=[SearchAlgo.CONTINUOUS, SearchAlgo.CONCURRENT],
    layout={"width": "max-content"},
    description="Algorithm configuration:",
)

winlen_widget = widgets.IntSlider(
    value=3,
    min=1,
    max=24,
    step=1,
    description="weather window length (hrs):",
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

oplen_widget = widgets.IntSlider(
    value=3,
    min=1,
    max=24,
    step=1,
    description="Nominal length of operation (hrs):",
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)

computation_mode_widget = widgets.RadioButtons(
    options=[ComputationMode.EACH_MONTH, ComputationMode.SPECIFIC_DATE],
    layout={"width": "max-content"},
    description="Computation mode:",
)

datetime_widget = NaiveDatetimePicker(
    description="Pick a Date",
    value=datetime.fromisoformat(client.config.get("default", "min-start-date")),
    continuous_update=False,
)

startday_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=31,
    step=1,
    description="Starting day",
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)


def update_operation_plot_widgets(event=None):
    with output_operation_plot_widgets:
        clear_output()
        if computation_mode_widget.value == ComputationMode.EACH_MONTH:
            display(startday_widget)
        else:
            display(datetime_widget)
    update_operation_plot(event)


url_widget.observe(update_dataframe, "value")
criteria_widget.observe(update_plots, "value")
concurrent_windows_widget.observe(update_plots, "value")
winlen_widget.observe(update_weather_window_plot, "value")
oplen_widget.observe(update_operation_plot, "value")
computation_mode_widget.observe(update_operation_plot_widgets, "value")
datetime_widget.observe(update_operation_plot, "value")
startday_widget.observe(update_operation_plot, "value")

In [ ]:
flex_orientation = "rows"

## Configurations

### Configuration

In [ ]:
display(url_widget)
display(criteria_widget)
display(concurrent_windows_widget)
display(output)

## Computation

### Weather window

In [ ]:
display(winlen_widget)
display(output_weather_window_plot)

In [ ]:
def plot_wwmonstats(data, winlen, concurrent_windows):
    windetect = ww_calc(data, winlen=winlen, concurrent_windows=concurrent_windows)
    results = wwmonstats(windetect)
    stats = results.describe(percentiles=percentiles).transpose()

    fig = make_subplots(rows=2, cols=1)
    for colname in stats.columns:
        fig.append_trace(
            go.Scatter(
                x=[MONTH_NAMES[i] for i in stats.index],
                y=stats[colname],
                name=colname,
                legendgroup="by_month",
            ),
            row=1,
            col=1,
        )

    for month_idx in results.columns:
        fig.append_trace(
            go.Scatter(
                x=results.index,
                y=results[month_idx],
                name=MONTH_NAMES[month_idx],
                legendgroup="by_year",
            ),
            row=2,
            col=1,
        )

    fig.update_xaxes(title_text="Month", row=1, col=1)
    fig.update_yaxes(title_text="Number of Weather Window", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Number of Weather Window", row=2, col=1)
    fig.update_layout(
        height=800,
        legend_tracegroupgap=130,
        title=f"Monthly statistics of number of weather window ({concurrent_windows_widget.value})",
    )
    return fig

### Operational planning

In [ ]:
display(oplen_widget)
display(computation_mode_widget)
display(output_operation_plot_widgets)
display(output_operation_plot)

update_operation_plot_widgets()

In [ ]:
def plot_olmonstats(data, oplen, critical_operation, date, monstrt):
    oplendetect = oplen_calc(
        data,
        oplen=oplen,
        critical_operation=critical_operation,
        date=date,
        monstrt=monstrt,
    )
    results = olmonstats(oplendetect)
    stats = results.describe(percentiles=percentiles).transpose()

    fig = make_subplots(rows=2, cols=1)
    for colname in stats.columns:
        fig.append_trace(
            go.Scatter(
                x=[MONTH_NAMES[i] for i in stats.index],
                y=stats[colname],
                name=colname,
                legendgroup="by_month",
            ),
            row=1,
            col=1,
        )

    for month_idx in results.columns:
        fig.append_trace(
            go.Scatter(
                x=results.index,
                y=results[month_idx],
                name=MONTH_NAMES[month_idx],
                legendgroup="by_year",
            ),
            row=2,
            col=1,
        )

    fig.update_xaxes(title_text="Month", row=1, col=1)
    fig.update_yaxes(title_text="Number of Weather Window", row=1, col=1)
    fig.update_xaxes(title_text="Year", row=2, col=1)
    fig.update_yaxes(title_text="Number of Weather Window", row=2, col=1)
    fig.update_layout(
        height=800,
        legend_tracegroupgap=130,
        title=f"Monthly statistics of number of weather window ({concurrent_windows_widget.value})",
    )
    return fig